# متن کی درجہ بندی کا کام

جیسا کہ ہم نے ذکر کیا ہے، ہم ایک سادہ متن کی درجہ بندی کے کام پر توجہ مرکوز کریں گے جو **AG_NEWS** ڈیٹاسیٹ پر مبنی ہے۔ اس کا مقصد خبروں کی سرخیوں کو چار زمروں میں تقسیم کرنا ہے: عالمی، کھیل، کاروبار، اور سائنس/ٹیکنالوجی۔

## ڈیٹاسیٹ

یہ ڈیٹاسیٹ [`torchtext`](https://github.com/pytorch/text) ماڈیول میں شامل ہے، اس لیے ہم اسے آسانی سے حاصل کر سکتے ہیں۔


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

یہاں، `train_dataset` اور `test_dataset` میں مجموعے شامل ہیں جو بالترتیب لیبل (کلاس کا نمبر) اور متن کے جوڑے واپس کرتے ہیں، مثال کے طور پر:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

تو، آئیے اپنے ڈیٹاسیٹ سے پہلی 10 نئی سرخیاں پرنٹ کریں:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

چونکہ ڈیٹاسیٹس اٹریٹرز ہیں، اگر ہم ڈیٹا کو متعدد بار استعمال کرنا چاہتے ہیں تو ہمیں اسے فہرست میں تبدیل کرنا ہوگا:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## ٹوکنائزیشن

اب ہمیں متن کو **نمبرز** میں تبدیل کرنے کی ضرورت ہے تاکہ انہیں ٹینسرز کے طور پر ظاہر کیا جا سکے۔ اگر ہم لفظی سطح کی نمائندگی چاہتے ہیں، تو ہمیں دو کام کرنے ہوں گے:
* **ٹوکنائزر** کا استعمال کرتے ہوئے متن کو **ٹوکنز** میں تقسیم کریں
* ان ٹوکنز کا ایک **لغت** بنائیں۔


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

لفظیات کا استعمال کرتے ہوئے، ہم آسانی سے اپنے ٹوکنائزڈ سٹرنگ کو نمبروں کے ایک سیٹ میں انکوڈ کر سکتے ہیں:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## الفاظ کے تھیلے کی متن کی نمائندگی

چونکہ الفاظ معنی کی نمائندگی کرتے ہیں، بعض اوقات ہم صرف انفرادی الفاظ کو دیکھ کر متن کا مطلب سمجھ سکتے ہیں، چاہے جملے میں ان کی ترتیب کچھ بھی ہو۔ مثال کے طور پر، خبروں کی درجہ بندی کرتے وقت، الفاظ جیسے *موسم*، *برف* ممکنہ طور پر *موسمی پیش گوئی* کی طرف اشارہ کریں گے، جبکہ الفاظ جیسے *حصص*، *ڈالر* *مالی خبروں* کی طرف شمار ہوں گے۔

**الفاظ کے تھیلے** (BoW) ویکٹر کی نمائندگی سب سے زیادہ استعمال ہونے والی روایتی ویکٹر نمائندگی ہے۔ ہر لفظ کو ایک ویکٹر انڈیکس سے جوڑا جاتا ہے، اور ویکٹر عنصر کسی دیے گئے دستاویز میں کسی لفظ کے وقوعات کی تعداد کو ظاہر کرتا ہے۔

![تصویر جو دکھاتی ہے کہ الفاظ کے تھیلے کی ویکٹر نمائندگی میموری میں کیسے ظاہر کی جاتی ہے۔](../../../../../translated_images/ur/bag-of-words-example.606fc1738f1d7ba9.webp)

> **نوٹ**: آپ BoW کو متن میں انفرادی الفاظ کے لیے تمام ایک-ہاٹ-انکوڈڈ ویکٹرز کے مجموعے کے طور پر بھی سوچ سکتے ہیں۔

نیچے ایک مثال دی گئی ہے کہ Scikit Learn پائتھون لائبریری کا استعمال کرتے ہوئے الفاظ کے تھیلے کی نمائندگی کیسے بنائی جائے:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

اے جی_نیوز ڈیٹاسیٹ کی ویکٹر نمائندگی سے بیگ آف ورڈز ویکٹر کا حساب لگانے کے لیے، ہم درج ذیل فنکشن استعمال کر سکتے ہیں:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **نوٹ:** یہاں ہم عالمی `vocab_size` متغیر استعمال کر رہے ہیں تاکہ لغت کے ڈیفالٹ سائز کو متعین کیا جا سکے۔ چونکہ اکثر لغت کا سائز کافی بڑا ہوتا ہے، ہم لغت کے سائز کو سب سے زیادہ عام الفاظ تک محدود کر سکتے ہیں۔ `vocab_size` کی قدر کو کم کرنے کی کوشش کریں اور نیچے دیے گئے کوڈ کو چلائیں، اور دیکھیں کہ یہ درستگی پر کیسے اثر ڈالتا ہے۔ آپ کو کچھ درستگی میں کمی کی توقع کرنی چاہیے، لیکن کارکردگی میں اضافے کے بدلے یہ کمی زیادہ نہیں ہوگی۔


## بو ڈبلیو کلاسیفائر کی تربیت

اب جب کہ ہم نے یہ سیکھ لیا ہے کہ اپنے متن کا بیگ آف ورڈز نمائندگی کیسے بنائیں، آئیے اس پر ایک کلاسیفائر تربیت کریں۔ سب سے پہلے، ہمیں اپنے ڈیٹاسیٹ کو تربیت کے لیے اس طرح تبدیل کرنا ہوگا کہ تمام پوزیشنل ویکٹر نمائندگیاں بیگ آف ورڈز نمائندگی میں تبدیل ہو جائیں۔ یہ اس طرح ممکن ہے کہ `bowify` فنکشن کو معیاری ٹارچ `DataLoader` میں `collate_fn` پیرامیٹر کے طور پر پاس کیا جائے:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

آئیے اب ایک سادہ کلاسفائر نیورل نیٹ ورک کی تعریف کرتے ہیں جو ایک لکیری پرت پر مشتمل ہے۔ ان پٹ ویکٹر کا سائز `vocab_size` کے برابر ہے، اور آؤٹ پٹ سائز کلاسز کی تعداد (4) کے مطابق ہے۔ چونکہ ہم درجہ بندی کا کام حل کر رہے ہیں، آخری ایکٹیویشن فنکشن `LogSoftmax()` ہے۔


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

اب ہم معیاری PyTorch تربیتی لوپ کی وضاحت کریں گے۔ چونکہ ہمارا ڈیٹا سیٹ کافی بڑا ہے، تدریسی مقصد کے لیے ہم صرف ایک ایپوک کے لیے تربیت کریں گے، اور کبھی کبھار ایک ایپوک سے بھی کم ( `epoch_size` پیرامیٹر کی وضاحت کرنے سے ہمیں تربیت کو محدود کرنے کی اجازت ملتی ہے)۔ ہم تربیت کے دوران جمع شدہ تربیتی درستگی کی بھی رپورٹ کریں گے؛ رپورٹنگ کی تعدد `report_freq` پیرامیٹر کا استعمال کرتے ہوئے وضاحت کی جاتی ہے۔


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## بائی گرامز، ٹرائی گرامز اور این-گرامز

الفاظ کے تھیلے کے طریقے کی ایک محدودیت یہ ہے کہ کچھ الفاظ کئی الفاظ پر مشتمل اظہار کا حصہ ہوتے ہیں، مثال کے طور پر، لفظ 'ہاٹ ڈاگ' کا مطلب بالکل مختلف ہوتا ہے 'ہاٹ' اور 'ڈاگ' کے دوسرے سیاق و سباق میں۔ اگر ہم ہمیشہ الفاظ 'ہاٹ' اور 'ڈاگ' کو ایک ہی ویکٹرز کے ذریعے ظاہر کریں، تو یہ ہمارے ماڈل کو الجھا سکتا ہے۔

اس مسئلے کو حل کرنے کے لیے، **این-گرام کی نمائندگی** اکثر دستاویز کی درجہ بندی کے طریقوں میں استعمال کی جاتی ہے، جہاں ہر لفظ، دو الفاظ یا تین الفاظ کی فریکوئنسی کو کلاسیفائرز کی تربیت کے لیے ایک مفید خصوصیت سمجھا جاتا ہے۔ بائی گرام کی نمائندگی میں، مثال کے طور پر، ہم اصل الفاظ کے علاوہ تمام لفظی جوڑوں کو بھی ذخیرہ الفاظ میں شامل کریں گے۔

نیچے ایک مثال دی گئی ہے کہ کس طرح Scikit Learn کا استعمال کرتے ہوئے بائی گرام الفاظ کے تھیلے کی نمائندگی تیار کی جا سکتی ہے:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-گرام طریقہ کار کا سب سے بڑا نقصان یہ ہے کہ الفاظ کی تعداد بہت تیزی سے بڑھنے لگتی ہے۔ عملی طور پر، ہمیں N-گرام کی نمائندگی کو کچھ جہتی کمی کی تکنیکوں کے ساتھ جوڑنے کی ضرورت ہوتی ہے، جیسے *ایمبیڈنگز*، جن پر ہم اگلے یونٹ میں بات کریں گے۔

**AG News** ڈیٹا سیٹ میں N-گرام کی نمائندگی استعمال کرنے کے لیے، ہمیں خاص ngram الفاظ کی فہرست بنانی ہوگی:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


ہم اوپر دیے گئے کوڈ کو استعمال کرتے ہوئے classifier کو ٹرین کر سکتے ہیں، لیکن یہ طریقہ میموری کے لحاظ سے بہت غیر مؤثر ہوگا۔ اگلے یونٹ میں، ہم bigram classifier کو embeddings کے ذریعے ٹرین کریں گے۔

> **نوٹ:** آپ صرف وہی ngrams چھوڑ سکتے ہیں جو متن میں مخصوص تعداد سے زیادہ بار ظاہر ہوں۔ اس سے یہ یقینی ہوگا کہ کم ظاہر ہونے والے bigrams کو نظرانداز کیا جائے گا، اور dimensionality میں نمایاں کمی آئے گی۔ اس کے لیے، `min_freq` پیرامیٹر کو ایک زیادہ قدر پر سیٹ کریں، اور vocabulary کی لمبائی میں تبدیلی کا مشاہدہ کریں۔


## اصطلاح فریکوئنسی انورس ڈاکیومنٹ فریکوئنسی TF-IDF

BoW کی نمائندگی میں، الفاظ کی موجودگی کو یکساں وزن دیا جاتا ہے، چاہے وہ لفظ خود کتنا ہی اہم کیوں نہ ہو۔ تاہم، یہ واضح ہے کہ عام الفاظ جیسے *a*، *in* وغیرہ، خاص اصطلاحات کے مقابلے میں درجہ بندی کے لیے کم اہم ہیں۔ حقیقت میں، زیادہ تر NLP کاموں میں کچھ الفاظ دوسروں کے مقابلے میں زیادہ اہمیت رکھتے ہیں۔

**TF-IDF** کا مطلب ہے **اصطلاح فریکوئنسی–انورس ڈاکیومنٹ فریکوئنسی**۔ یہ الفاظ کے بیگ کی ایک قسم ہے، جہاں کسی دستاویز میں کسی لفظ کی موجودگی کو ظاہر کرنے کے لیے 0/1 کی بائنری ویلیو کے بجائے ایک فلوٹنگ پوائنٹ ویلیو استعمال کی جاتی ہے، جو کارپس میں لفظ کی موجودگی کی فریکوئنسی سے متعلق ہوتی ہے۔

زیادہ رسمی طور پر، کسی لفظ $i$ کا وزن $w_{ij}$ کسی دستاویز $j$ میں درج ذیل طور پر بیان کیا جاتا ہے:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
جہاں
* $tf_{ij}$ کسی دستاویز $j$ میں لفظ $i$ کی موجودگی کی تعداد ہے، یعنی وہ BoW ویلیو جو ہم پہلے دیکھ چکے ہیں
* $N$ مجموعی دستاویزات کی تعداد ہے
* $df_i$ وہ دستاویزات کی تعداد ہے جن میں لفظ $i$ پورے مجموعے میں موجود ہے

TF-IDF ویلیو $w_{ij}$ کسی دستاویز میں کسی لفظ کے ظاہر ہونے کی تعداد کے تناسب سے بڑھتی ہے اور کارپس میں موجود دستاویزات کی تعداد کے ذریعے ایڈجسٹ کی جاتی ہے جن میں وہ لفظ موجود ہے۔ یہ اس حقیقت کو ایڈجسٹ کرنے میں مدد کرتا ہے کہ کچھ الفاظ دوسروں کے مقابلے میں زیادہ بار ظاہر ہوتے ہیں۔ مثال کے طور پر، اگر کوئی لفظ *ہر* دستاویز میں موجود ہو، تو $df_i=N$ ہوگا، اور $w_{ij}=0$ ہوگا، اور وہ اصطلاحات مکمل طور پر نظرانداز کر دی جائیں گی۔

آپ Scikit Learn کا استعمال کرتے ہوئے آسانی سے متن کی TF-IDF ویکٹرائزیشن بنا سکتے ہیں:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## نتیجہ

اگرچہ TF-IDF نمائندگی مختلف الفاظ کو فریکوئنسی وزن فراہم کرتی ہے، یہ معنی یا ترتیب کو ظاہر کرنے سے قاصر ہے۔ جیسا کہ مشہور لسانیات دان جے آر فرث نے 1935 میں کہا تھا، "کسی لفظ کا مکمل مطلب ہمیشہ سیاق و سباق پر مبنی ہوتا ہے، اور سیاق و سباق کے بغیر معنی کا کوئی مطالعہ سنجیدگی سے نہیں لیا جا سکتا۔"۔ ہم اس کورس میں آگے چل کر سیکھیں گے کہ زبان ماڈلنگ کا استعمال کرتے ہوئے متن سے سیاق و سباق کی معلومات کو کیسے حاصل کیا جا سکتا ہے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
